In [2]:
# Compare Algorithms
from pandas import read_csv
from matplotlib import pyplot
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# load data"set
filename = "train.csv"
names = ["PassId","Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
dataframe = read_csv(filename, names=names, dtype={"":np.int32}, header=0, \
    usecols = ["Survived", "Pclass","SibSp", "Parch"])
array = dataframe.values
#, "Sex"  , "Age"  , converters = {"Sex":lambda x: 1 if x == "male" else 0}
X = array[:,1:]
Y = array[:,0]
# prepare models
models = []
models.append(("LR", LogisticRegression()))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("CART", DecisionTreeClassifier()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))
# evaluate each model in turn
results = []
names = []
scoring = "accuracy"
for name, model in models:
  kfold = KFold(n_splits=15, random_state=7)
  cv_results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
  results.append(cv_results)                                                    
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

LR: 0.687137 (0.056694)
LDA: 0.692731 (0.053979)
KNN: 0.660640 (0.124114)
CART: 0.701827 (0.068486)
NB: 0.663672 (0.068021)
SVM: 0.706290 (0.075237)


In [3]:
# Stochastic Gradient Boosting Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from pickle import dump
from pickle import load
# load data"set
filename = "train.csv"
names = ["PassId","Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
dataframe = read_csv(filename, names=names, dtype={"":np.int32}, header=0, \
    usecols = ["Survived", "Pclass","SibSp", "Parch"])
array = dataframe.values
#, "Sex"  , converters = {"Sex":lambda x: 1 if x == "male" else 0}
X = array[:,1:]
Y = array[:,0]
seed = 7
num_trees = 100
kfold = KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed) 

filename =  "finalized_model.csv"
dump(model, open(filename,"wb"))
loaded_model = load(open(filename,"rb"))

results = cross_val_score(loaded_model, X, Y, cv=kfold)
print(results.mean())

0.7083021223470662


In [5]:
import pandas
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
filename = "train.csv"
names = ["PassId","Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
dataframe = read_csv(filename, names=names, dtype={"":np.int32}, header=0, \
    usecols = ["Survived", "Pclass","SibSp", "Parch"])
array = dataframe.values
#, "Sex"  , converters = {"Sex":lambda x: 1 if x == "male" else 0}
X = array[:,1:]
Y = array[:,0]
kfold = KFold(n_splits=10, random_state=7)
model = SVC()
results = cross_val_score(model, X, Y, cv=kfold)
model.fit(X,Y)

filename =  "finalized_model.csv"
dump(model, open(filename,"wb"))
loaded_model = load(open(filename,"rb"))
filename2 = "test_Class_Sib_Parch.csv"
names = ["PassId","Pclass","SibSp", "Parch"]
dataframe = read_csv(filename2, names=names, header=0)
array = dataframe.values
X = array[:,1:]
predictions = loaded_model.predict(X)
prediction = pandas.DataFrame(predictions, columns=['Survived'])#.to_csv('prediction.csv')
filename3 = "submissionTemplate.csv"
subDf = read_csv(filename3)
subDf['Survived'] = prediction['Survived']
subDf.to_csv("submission.csv")